## Convolution 1D


In [1]:
import numpy as np
def conv1d(x, w, p=0,s=1):
    w_rot=np.array(w[::-1])
    x_padded=np.array(x)
    if p>0:
        zero_pad=np.zeros(shape=p)
        x_padded=np.concatenate([zero_pad,x_padded,zero_pad])
    res=[]
    for i in range(0,int((len(x_padded)-len(w_rot))/s)+1,s):
        res.append(np.sum(x_padded[i:i+w_rot.shape[0]]*w_rot))
    return np.array(res)
x=np.array([1,3,2,4,5,6,1,3])
w=np.array([1,0,3,1,2])

print('Conv1d Implementation:',conv1d(x,w,p=2,s=1))
print('Numpy Results:',np.convolve(x,w,mode='same'))


Conv1d Implementation: [ 5. 14. 16. 26. 24. 34. 19. 22.]
Numpy Results: [ 5 14 16 26 24 34 19 22]


## CNN on MNIST


In [15]:
import torch
import torchvision
from torchvision import transforms


image_path="../"
transforms=transforms.Compose([transforms.ToTensor()])
mnist_dataset=torchvision.datasets.MNIST(root=image_path,train=True,transform=transforms,download=False)
from torch.utils.data import Subset
mnist_valid_dataset = Subset(mnist_dataset, torch.arange(10000))
mnist_train_dataset = Subset(mnist_dataset, torch.arange(10000, len(mnist_dataset)))
mnist_test_dataset = torchvision.datasets.MNIST(root=image_path, train=False, transform=transforms, download=False)



In [16]:
from torch import manual_seed
from torch.utils.data import DataLoader
batch_size = 64
torch,manual_seed(1)
train_dl = DataLoader(mnist_train_dataset, batch_size, shuffle=True)
valid_dl = DataLoader(mnist_valid_dataset, batch_size, shuffle=False)


### Constructing CNN using Pytorch


In [27]:
import torch.nn as nn
model = nn.Sequential()
model.add_module('conv1', nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding=2 ))
model.add_module('relu1', nn.ReLU())
model.add_module('pool1', nn.MaxPool2d(kernel_size=2))
model.add_module(
    'conv2', nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5,padding=2))
model.add_module('relu2', nn.ReLU())
model.add_module('pool2', nn.MaxPool2d(kernel_size=2))
model.add_module('flatten', nn.Flatten())
model.add_module('fc1', nn.Linear(3136, 1024))
model.add_module('relu3', nn.ReLU())
model.add_module('dropout', nn.Dropout(0.5))
model.add_module('fc2', nn.Linear(1024, 10))

In [18]:

x = torch.ones((4,1,28,28))
model(x).shape

torch.Size([4, 10])

In [28]:
loss_fn=nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.parameters(),lr=0.001)


In [29]:
# training the model
def train(model, num_epochs, train_dl,valid_dl):
    loss_hist_train= [0]*num_epochs
    accuracy_hist_train=[0]*num_epochs
    loss_hist_valid=[0]*num_epochs
    accuracy_hist_valid=[0]*num_epochs
    for epoch in range(num_epochs):
        model.train()   
        for x_batch,y_batch in train_dl:
            y_pred=model(x_batch)
            loss=loss_fn(y_pred,y_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss_hist_train[epoch]+=loss.item()*y_batch.size(0)
            is_correct=(torch.argmax(y_pred,dim=1)==y_batch).float()
            accuracy_hist_train[epoch]+=is_correct.sum()
        loss_hist_train[epoch]=loss_hist_train[epoch]/len(train_dl.dataset)
        accuracy_hist_train[epoch]=accuracy_hist_train[epoch]/len(train_dl.dataset)
    model.eval()
    with torch.no_grad():
        for x_batch,y_batch in valid_dl:
            y_pred=model(x_batch)
            loss=loss_fn(y_pred,y_batch)
            loss_hist_valid[epoch]=loss.item()*y_batch.size(0)
            is_correct=(torch.argmax(y_pred,dim=1)==y_batch).float()
            accuracy_hist_valid[epoch]=is_correct.sum()
    loss_hist_valid[epoch]=loss_hist_valid[epoch]/len(valid_dl.dataset)
    accuracy_hist_valid[epoch]=accuracy_hist_valid[epoch]/len(valid_dl.dataset)
    print(f'Epoch {epoch+1}/{num_epochs},train_loss: {loss_hist_train[epoch]:.4f},train_accuracy: {accuracy_hist_train[epoch]:.4f},valid_loss: {loss_hist_valid[epoch]:.4f},valid_accuracy: {accuracy_hist_valid[epoch]:.4f}')
    return loss_hist_train,accuracy_hist_train,loss_hist_valid,accuracy_hist_valid
    


In [30]:
torch.manual_seed(1)
num_epochs=20
hist=train(model,num_epochs,train_dl,valid_dl)

Epoch 20/20,train_loss: 0.0073,train_accuracy: 0.9978,valid_loss: 0.0000,valid_accuracy: 0.0016


In [33]:
torch.save(model, "model-cnn.pth")